# 16. Serialization

Serialization of an object means that we are converting it to its binary form. Vice versa, deserialization of an object is to convert it from binary data back into an object. Most often it used when we need to write an object to file, then reading it back from the file. To serialize objects into files, we need to use two types of readers: `FileStream` to deal with the file and `ObjectStream` to deal with the object. This is similar to read/write strings where we have the `FileReader()` object and the `BufferedReader()` object.  

In [15]:
import java.io.FileOutputStream;
import java.io.ObjectOutputStream;
import java.io.FileInputStream;
import java.io.ObjectInputStream;
import java.io.IOException;
import java.io.FileNotFoundException;
import java.io.Serializable;

In order to be able to serialize an object, we need to implement the `Serializable` interface. When we implement this interface, Java will give warning about not having a `SerialVersionUID`. Implementing the static final variable will get rid of the warning. This variable is a safety catch to ensure that it is the same object being written and read from file. If the UID is different in the program that read the File, it will throw an error about not retrieving the same object. One common usage of the `SerialVersionUID` is to handle different versions of the program, especially in the cases where objects and features may become incompatible between versions. 

In [5]:
public class Person implements Serializable {
    
    private static final long serialVersionUID = 4801633;
    
    private int id;
    private String name;
    
    public Person(int id, String name) {
        this.id = id;
        this.name = name;
    }
    
    @Override
    public String toString() {
        return "Person [id=" + id + ", name=" + name +"]";
    }
}

Writing the serializable `Person` object to file. We will need to use a `FileOutputStream()` and a `ObjectOutputStream()` object.

In [23]:
public class WriteObjects {
    public static void main() {
        System.out.println("Writing objects... ");
        
        Person mike = new Person(543, "Mike");
        Person sue = new Person(123, "Sue");
        
        System.out.println(mike);
        System.out.println(sue);
        
        try(FileOutputStream fs = new FileOutputStream("data/people.bin")) {
            
            ObjectOutputStream os = new ObjectOutputStream(fs);
            os.writeObject(mike);
            os.writeObject(sue);
            os.close();
            
        } catch (FileNotFoundException e) {
            // Can't create the file
            e.printStackTrace();
        } catch (IOException e) {
            // Can't write to the file
            e.printStackTrace();
        }
    }
}

In [24]:
WriteObjects.main();

Writing objects... 
Person [id=543, name=Mike]
Person [id=123, name=Sue]


Reading the person objects from the files back into the program. We will need to use the `FileInputStream()` and the `ObjectInputStream()` object. Further more, while streaming in the object, we need to cast the object read from the file back into the `Person()` object. 

In [25]:
public class ReadObjects {
    public static void main() {
        System.out.println("Reading objects... ");
        
        try(FileInputStream fi = new FileInputStream("data/people.bin")) {
            
            ObjectInputStream os = new ObjectInputStream(fi);
            
            // Need to cast the object read into the Person object
            Person person1 = (Person)os.readObject(); 
            Person person2 = (Person)os.readObject();
            os.close();
            
            System.out.println(person1);
            System.out.println(person2);

        // Can't find file        
        } catch (FileNotFoundException e) {
            e.printStackTrace();
            
        // Can't read file
        } catch (IOException e) {
            e.printStackTrace();
        
        // Cannot cast the object read from file into the
        // Person object. Or reading an object that is not
        // defined in this program
        } catch(ClassNotFoundException e) {
            e.printStackTrace();
        }
    }
}

In [26]:
ReadObjects.main();

Reading objects... 
Person [id=543, name=Mike]
Person [id=123, name=Sue]


> **NOTE**: the constructors are not run when de-serializing objects. Objects are simply being retrieved from its binary form. 

## Serializing Array

An array in Java is also considered an object. So it is also possible to serialize arrays, as long as the objects within the array is serializable (The object implements the `Serializable` interface). 

In [25]:
public class WriteArrayObjects {
    public static void main() {
        System.out.println("Writing objects... ");
        
        // Array of objects
        Person[] people = {new Person(543, "Mike"), new Person(123, "Sue"), new Person(7, "Bob")};
        
        // Array Lists => Will be covered in the Java Collections
        ArrayList<Person> peopleList = new ArrayList<Person>(Arrays.asList(people));
        
        try(FileOutputStream fs = new FileOutputStream("data/people.ser")) {
            
            ObjectOutputStream os = new ObjectOutputStream(fs);
            
            // Write Array
            os.writeObject(people);
            
            // Write ArrayList
            os.writeObject(peopleList);
            
            // Write objects one-by-one
            os.writeInt(peopleList.size()); // Write the length of the list
            for (Person person: peopleList) {
                os.writeObject(person);
            }

            os.close();
            
        } catch (FileNotFoundException e) {
            // Can't create the file
            e.printStackTrace();
        } catch (IOException e) {
            // Can't write to the file
            e.printStackTrace();
        }
    }
}

In [26]:
WriteArrayObjects.main();

Writing objects... 


In [32]:
public class ReadArrayObjects {
    public static void main() {
        System.out.println("Reading objects... ");
        
        try(FileInputStream fi = new FileInputStream("data/people.ser")) {
            
            ObjectInputStream os = new ObjectInputStream(fi);
            
            // Need to cast the object read into the Person Array object
            Person[] people = (Person[])os.readObject();
            ArrayList<Person> peopleList = (ArrayList<Person>)os.readObject();
            
            for (Person person: people) {
                System.out.println(person);
            }
            
            for (Person person: peopleList) {
                System.out.println(person);
            }
            
            // Reading the objects written one-by-one
            int num = os.readInt();
            for (int i=0; i<num; i++){
                Person person = (Person)os.readObject();
                System.out.println(person);
            }
            os.close();

        // Can't find file        
        } catch (FileNotFoundException e) {
            e.printStackTrace();
            
        // Can't read file
        } catch (IOException e) {
            e.printStackTrace();
        
        // Cannot cast the object read from file into the
        // Person object. Or reading an object that is not
        // defined in this program
        } catch(ClassNotFoundException e) {
            e.printStackTrace();
        }
    }
}

In [31]:
ReadArrayObjects.main();

Reading objects... 
Person [id=543, name=Mike]
Person [id=123, name=Sue]
Person [id=7, name=Bob]
Person [id=543, name=Mike]
Person [id=123, name=Sue]
Person [id=7, name=Bob]
Person [id=543, name=Mike]
Person [id=123, name=Sue]
Person [id=7, name=Bob]


## Transient Keyword

The `transient` keyword is used to indicate information that we do not want to serialize in a `Serializable` object. To demonstrate the `transient` keyword function we will set the `id` variable as transient in the TransientPerson object. 

A similar case is seen with `static` variables, because these variables are not associated with the instances, and often it is not reasonable to serialize them. 

In [49]:
public class TransientPerson implements Serializable {
    
    private static final long serialVersionUID = 4801633;
    
    private transient int id; // transient variable
    private String name;
    private static int count;
    
    public TransientPerson(int id, String name) {
        this.id = id;
        this.name = name;
    }
    
    public static int getCount(){
        return count;
    }
    
    public static void setCount(int count){
        TransientPerson.count = count;
    }
    
    @Override
    public String toString() {
        return "Person [id=" + id + ", name=" + name + ", count=" + count +"]";
    }
}

In [50]:
public class WriteTransObjects {
    public static void main() {
        System.out.println("Writing objects... ");
        
        TransientPerson mike = new TransientPerson(543, "Mike");
        TransientPerson.setCount(8);
        
        System.out.println(mike);
        
        // A tidier way to initiate both output streams with try-with-resources
        try(ObjectOutputStream os = new ObjectOutputStream(new FileOutputStream("data/transientpeople.bin"))) {
            os.writeObject(mike);
            
        } catch (FileNotFoundException e) {
            // Can't create the file
            e.printStackTrace();
        } catch (IOException e) {
            // Can't write to the file
            e.printStackTrace();
        }
    }
}

In [51]:
WriteTransObjects.main();

Writing objects... 
Person [id=543, name=Mike, count=8]


In [52]:
public class ReadTransObjects {
    public static void main() {
        System.out.println("Reading objects... ");
        
        // A tidier way to initiate both input streams with try-with-resources
        try(ObjectInputStream os = new ObjectInputStream(new FileInputStream("data/transientpeople.bin"))) {
            
            // Need to cast the object read into the Person object
            TransientPerson person1 = (TransientPerson)os.readObject();             
            System.out.println(person1);

        // Can't find file        
        } catch (FileNotFoundException e) {
            e.printStackTrace();
            
        // Can't read file
        } catch (IOException e) {
            e.printStackTrace();
        
        // Cannot cast the object read from file into the
        // Person object. Or reading an object that is not
        // defined in this program
        } catch(ClassNotFoundException e) {
            e.printStackTrace();
        }
    }
}

In [53]:
ReadTransObjects.main();

Reading objects... 
Person [id=0, name=Mike, count=8]


**For the `transient` variable**: We can see that in after writing and reading the object, the `id` is lost in the process. In Java Objects, the attributes are initiated with default values. `int` default value is `0`, and `String` is `null`. We can see that when reading the object from file, the `id` returned to its default value. 
**For `static` instances**: These variables are not serialized by default. 